# Q Learning Example

此程式碼將用 tabular Q-learning 的方法實現一個小例子，例子的環境是一個一維的世界，在世界的右邊有寶藏，探索者只要得到寶藏嚐到了甜頭，然後以後就記住了得到寶藏的方法，這就是他用強化學習所學習到的行為

### -o---T
    T：寶藏的位置
    o：探索者的位置

Q-learning 是一種記錄行為值（Q value）的方法，每種在一定狀態的行為都會有一個值 Q(s, a)，就是說行為 a 在狀態 s 的值是 Q(s, a)。s 在上面的探索者遊戲中，就是 o 所在的地點了。而每一個地點探索者都能做出兩種行為 left、right ，這就是探索者的所有可行的 a。

如果在某個地點 s1，探索者計算了他能有的兩個行為，a1, a2 = left, right，計算結果是 Q(s1, a1) > Q(s1, a2)，那麼探索者就會選擇 left 這個行為，這就是 Q-learning 的行為選擇簡單規則。

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
N_STATES = 6                 # 一維世界的長度
ACTIONS  = ['left', 'right'] # 探索者可用的動作
EPSILON  = 0.9               # 貪婪度 greedy
ALPHA    = 0.1               # 學習率
GAMMA    = 0.9               # 獎勵遞減值
MAX_EPISODES = 13            # 最大回合數
FRESH_TIME = 0.3             # 移動的間隔時間

### Q table
    對於 tabular Q learning，我們必須將所有的 Q values（行為值）放在 q_table 中，更新 q_table 也是在更新他的行為準則。q_table 的 index 是對所有對應的 state（探索者的位置），columns 是對應的 action（探索者行為）。

In [3]:
def build_Q_table(n_states, actions):
    table = pd.DataFrame(np.zeros((N_STATES, len(ACTIONS))), columns=actions)
    return table

# Q table
"""
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
"""

'\n   left  right\n0   0.0    0.0\n1   0.0    0.0\n2   0.0    0.0\n3   0.0    0.0\n4   0.0    0.0\n5   0.0    0.0\n'

### Define actions
    我們引入 epsilon greedy 的概念，因為在初始階段，隨機的探索環境，往往比固定的行為模式要好，所以這也是累積經驗的階段，我們希望探索者不會那麼貪婪（greedy），所以 EPSILON 就是用來控制貪婪程度的值。EPSILON 可以隨著探索時間不斷提升（越來越貪婪），不過在這個例子中，我們就固定成 EPSILON = 0.9，90% 的時間是選擇最優策略，10% 的時間來探索。

In [4]:
def choose_action(state, Q_table):
    state_actions = Q_table.iloc[state, :] # 選出這個 state 的所有 action 值
    if (np.random.uniform() > EPSILON) or (state_actions.all() == 0): # 非貪婪 or 這個 state 還沒探索過
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_actions.argmax()
    return action_name

### Environment feedback
    做出行為後，環境也要給我們的行為一個回饋，回饋出下一個 state (state_) 和在上一個 state 做出 action 所得到的 reward。這裡定義的規則就是，只有當 o 移動到了 T，探索者才會得到唯一的一個獎勵，獎勵值 reward = 1，其他情況都沒有獎勵。

In [5]:
def get_env_feedback(state, action):
    if action == 'right':         # move right
        if state == N_STATES - 2: # terminal
            state_ = 'terminal'
            reward = 1
        else:
            state_ = state + 1
            reward = 0
    else:                         # move left
        reward = 0
        if state == 0:
            state_ = state        # reach the wall
        else:
            state_ = state - 1
    return state_, reward

### Update environment

In [6]:
def update_env(state, episode, step_counter):
    env_list = ['-']*(N_STATES-1) + ['T']
    if state == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('{}'.format(interaction))
        time.sleep(2)
        print('', end='')
    else:
        env_list[state] = 'o'
        interaction = ''.join(env_list)
        print('{}'.format(interaction))
        time.sleep(FRESH_TIME)

### Q-learning algorithm

$Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha \cdot \left[ r_t + \gamma \max_a Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t) \right]$

In [10]:
def rl():
    q_table = build_Q_table(N_STATES, ACTIONS) # 初始 q table
    for episode in range(MAX_EPISODES):
        step_counter = 0
        state = 0           # 回合初始位置
        is_terminal = False # 是否結束回合
        update_env(state, episode, step_counter)  # 環境更新
        while not is_terminal:
            action = choose_action(state, q_table)           # 選行為
            state_, reward = get_env_feedback(state, action) # 實施行為並得到環境的回饋
            q_predict = q_table.loc[state, action]           # 估算的 Q value
            if state_ != 'terminal':
                q_target = reward + GAMMA * q_table.iloc[state_,:].max() # 實際的 Q value（回合尚未結束）
            else:
                q_target = reward # 實際的 Q value（回合結束）
                is_terminal = True
            
            q_table.loc[state, action] += ALPHA * (q_target - q_predict) # q table 更新
            state = state_ # 探索者移動到下一個 state
            update_env(state, episode, step_counter+1) # 環境更新
            print(q_table)
            
            step_counter += 1
    return q_table

In [11]:
q_table = rl()
print('\nQ table:\n')
print(q_table)

o----T
-o---T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
--o--T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
-o---T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
o----T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
-o---T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
o----T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
o----T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
-o---T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
--o--T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0


           left     right
0  5.904900e-07  0.000007
1  5.904900e-07  0.000073
2  0.000000e+00  0.004309
3  0.000000e+00  0.060192
4  2.268000e-03  0.343900
5  0.000000e+00  0.000000
-o---T
           left     right
0  5.904900e-07  0.000012
1  5.904900e-07  0.000073
2  0.000000e+00  0.004309
3  0.000000e+00  0.060192
4  2.268000e-03  0.343900
5  0.000000e+00  0.000000
--o--T
           left     right
0  5.904900e-07  0.000012
1  5.904900e-07  0.000453
2  0.000000e+00  0.004309
3  0.000000e+00  0.060192
4  2.268000e-03  0.343900
5  0.000000e+00  0.000000
---o-T
           left     right
0  5.904900e-07  0.000012
1  5.904900e-07  0.000453
2  0.000000e+00  0.009296
3  0.000000e+00  0.060192
4  2.268000e-03  0.343900
5  0.000000e+00  0.000000
--o--T
           left     right
0  5.904900e-07  0.000012
1  5.904900e-07  0.000453
2  0.000000e+00  0.009296
3  8.366004e-04  0.060192
4  2.268000e-03  0.343900
5  0.000000e+00  0.000000
-o---T
           left     right
0  5.904900e-07  0.000012
1  

           left     right
0  5.904900e-07  0.004086
1  1.162321e-05  0.022604
2  4.080942e-05  0.079951
3  8.366004e-04  0.278655
4  2.268000e-03  0.686189
5  0.000000e+00  0.000000
---o-T
           left     right
0  5.904900e-07  0.004086
1  1.162321e-05  0.022604
2  4.080942e-05  0.097035
3  8.366004e-04  0.278655
4  2.268000e-03  0.686189
5  0.000000e+00  0.000000
----oT
           left     right
0  5.904900e-07  0.004086
1  1.162321e-05  0.022604
2  4.080942e-05  0.097035
3  8.366004e-04  0.312547
4  2.268000e-03  0.686189
5  0.000000e+00  0.000000
Episode 12: total_steps = 5
           left     right
0  5.904900e-07  0.004086
1  1.162321e-05  0.022604
2  4.080942e-05  0.097035
3  8.366004e-04  0.312547
4  2.268000e-03  0.717570
5  0.000000e+00  0.000000
o----T
-o---T
           left     right
0  5.904900e-07  0.005712
1  1.162321e-05  0.022604
2  4.080942e-05  0.097035
3  8.366004e-04  0.312547
4  2.268000e-03  0.717570
5  0.000000e+00  0.000000
o----T
           left     right
0